<a href="https://colab.research.google.com/github/vivek09thakur/PINAKA/blob/main/PINAKA_Voice_Box_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!apt update && apt install -y espeak-ng # Install system dependencies
!pip install -U uv # Install UV package manager
!git clone https://github.com/Zyphra/Zonos.git # Clone Zonos repository
%cd Zonos
!uv sync   # Install dependencies using UV
!uv sync --extra compile
!pip install kanjize
!pip install phonemizer
!pip install SudachiPy
!pip install sudachidict_full

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
66 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as re

In [8]:
import torch
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
from IPython.display import Audio


class Kanth:
    def __init__(self, model_name):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device {self.device}")
        self.model_name = model_name
        self.model = Zonos.from_pretrained(self.model_name, device=self.device)
        self.speaker = None
        self.sampling_rate = None

    def load_sample(self, voice_sample, set_speed=421):
        """Load a reference voice sample and create speaker embedding"""
        print("Loading reference audio...")
        self.wave, self.sampling_rate = torchaudio.load(voice_sample)
        self.speaker = self.model.make_speaker_embedding(self.wave, self.sampling_rate)
        torch.manual_seed(set_speed)
        print(f"Speaker embedding created with seed {set_speed}")

    def generate_speech(self, text, language="en-us", speed=1.0, filename="output.wav"):
        """Generate speech from text"""
        if self.speaker is None:
            raise ValueError("Please load a voice sample first using load_sample()")

        print(f"Generating: {text}")

        # Create conditioning
        cond_dict = make_cond_dict(
            text=text,
            speaker=self.speaker,
            language=language,
            speed=speed
        )
        conditioning = self.model.prepare_conditioning(cond_dict)

        # Generate audio
        codes = self.model.generate(conditioning)
        wavs = self.model.autoencoder.decode(codes).cpu()

        # Save audio
        torchaudio.save(filename, wavs[0], self.model.autoencoder.sampling_rate)
        print(f"Audio saved as {filename}")
        return filename

    def play_audio(self, filename):
        """Play the generated audio"""
        return Audio(filename)